# Chuyển multi-label -> single-label sử dụng:
* Binary Relevance: coi mỗi nhãn là một phân loại lớp riêng biệt.
* Classifier Chains: bộ phân loại đầu tiên chỉ được huấn luyện dựa trên dữ liệu đầu vào và sau đó mỗi bộ phân loại tiếp theo được huấn luyện trên không gian đầu vào và tất cả các bộ phân loại trước đó trong chuỗi ⇒ Bảo toàn mối tương quan các nhãn.
* Label Powerset: chuyển BT thành BT đa lớp với một bộ phân loại đa lớp được đào tạo trên tất cả các kết hợp nhãn duy nhất được tìm thấy trong dữ liệu đào tạo.

In [1]:
!pip install scikit-multilearn regex

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
# ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report,f1_score

In [25]:
### Split Dataset into Train and Text
from sklearn.model_selection import train_test_split,cross_val_score,KFold
# Feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Multi Label Pkgs
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [6]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [7]:
# Load Dataset
# df = pd.read_csv("/kaggle/input/mydataset/data.csv")
# df = pd.read_csv("/kaggle/input/clean-data/clean_content_2.csv")

df = import_data("/kaggle/input/clean-data/clean_content_2.csv")

# Load Common Tags
# df_common_tags_freq = pd.read_csv("/kaggle/input/mydataset/common_tags.csv")
df_common_tags_freq = import_data("/kaggle/input/mydataset/common_tags.csv")
y = import_data("/kaggle/input/labeldata/multilabel.csv")

/tmp/ipykernel_193/147790395.py:42: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)


Memory usage of dataframe is 2.25 MB
Memory usage after optimization is: 9.42 MB
Decreased by -318.2%
Memory usage of dataframe is 1.15 MB
Memory usage after optimization is: 3.02 MB
Decreased by -162.4%


/tmp/ipykernel_193/147790395.py:42: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
/tmp/ipykernel_193/147790395.py:42: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)


Memory usage of dataframe is 2253.43 MB
Memory usage after optimization is: 281.68 MB
Decreased by 87.5%


In [8]:
df.head()

,content,tags,clean_content
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ...","La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,...",trận đấu barcelona nhanh chóng tràn tấn công v...
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ...",willian đi lịch sử bóng đá xứ samba bàn thắng ...
2,Giải vô địch ná cao su thế giới năm 2024 đã di...,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng...",giải vô địch ná cao su thế giới diễn hai thượn...
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục...",mục tiêu phó thủ tướng chính phủ lê thành long...
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...","toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề...",giải bơi lặn vô địch quốc gia kết thúc đà nẵng...


In [8]:
df_content = df[['content']]
df_tags = df[['tags']]

In [9]:
df_content.head()

,content
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ..."
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...
2,Giải vô địch ná cao su thế giới năm 2024 đã di...
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n..."


In [10]:
df_tags.head()

,tags
0,"La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,..."
1,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ..."
2,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng..."
3,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục..."
4,"toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề..."


In [11]:
df.dtypes

content    object
tags       object
dtype: object

In [12]:
df.loc[0].content

'Bước vào trận đấu, Barcelona nhanh chóng tràn lên tấn công nhưng vấp phải sự kháng cự quyết liệt của Sevilla. Bước ngoặt đến ở phút 22, Raphinha bị phạm lỗi trong vòng cấm và trọng tài đã cho Barcelona hưởng quả phạt đền. Trên chấm 11m, Lewandowski đã ghi bàn mở tỷ số cho Barcelona. Đến phút 28, Pedri đã có bàn nhân đôi cách biệt cho đội chủ nhà trước khi Lewandowski hoàn tất cú đúp cho riêng mình ở phút 39. Hiệp 1 khép lại với tỉ số 3-0 nghiêng về Barcelona. Sang hiệp 2, các chân sút của Barcelona liên tiếp bắn phá khung thành của Sevilla nhưng rất tiếc đều rơi vào thế việt vị. Tuy nhiên, chỉ trong vòng ít phút từ phút 82 đến 88, các khán giả có mặt trên sân Olimpic Lluis Companys đã được chứng kiến tới 3 bàn thắng. Cầu thủ vào sân thay người bên phía Barcelona là Torre lập cú đúp, xen giữa là bàn thắng danh dự của Idumbo bên phía Sevilla. Chung cuộc, Barca thắng Sevilal với tỷ số 5-1. Như vậy, đoàn quân của HLV Hansi Flick củng cố vị trí dẫn đầu trên bảng xếp hạng La Liga, duy trì k

In [13]:
df.loc[0].tags

'La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,Lewandowski,Barcelona,Raphinha,Pedri,Hansi Flick,tiếp đón,primera,củng cố vị trí,kháng cự,bắn phá,vào sân thay người,nhân đôi cách biệt,tỏa sáng,xen,sân nhà,chấm 11m'

In [8]:
df_common_tags_freq

,Tag,Frequency
0,năm,7588
1,Israel,2700
2,Liên bang Nga,2390
3,TP.HCM,2309
4,bão,2144
...,...,...
75529,Báo Quân khu,3
75530,Phổ Quang,3
75531,Nữ công tước,3
75532,Dương Văn Thắng,3


In [9]:
df_common_tags = df_common_tags_freq[['Tag']]
df_common_tags

,Tag
0,năm
1,Israel
2,Liên bang Nga
3,TP.HCM
4,bão
...,...
75529,Báo Quân khu
75530,Phổ Quang
75531,Nữ công tước
75532,Dương Văn Thắng


In [10]:
# Select 5000 common label
df_5000_tags = df_common_tags[0:5000]
df_5000_tags

,Tag
0,năm
1,Israel
2,Liên bang Nga
3,TP.HCM
4,bão
...,...
4995,chuyến đi
4996,tuyệt chủng
4997,Amazon
4998,Noussair Mazraoui


In [11]:
# Dataframe to list
list_5000_tags = df_5000_tags['Tag'].tolist()
# print(list_5000_tags[:10])

In [18]:
df_1 = df

# Text Preprocessing

In [ ]:
!pip install underthesea vncorenlp neattext

In [20]:
import underthesea
from underthesea import word_tokenize, text_normalize, pos_tag
import neattext as nt
from tqdm import tqdm
import regex as re 
tqdm.pandas()

In [21]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.readlines()
    # Clean stopwords (remove newline and extra spaces)
    stopwords = set([word.strip() for word in stopwords])
    return stopwords

stopwords = load_stopwords('/kaggle/input/mydataset/vietnamese-stopwords.txt')


def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]


def preprocess_text_vietnamese_with_stopwords(text):
    text_cleaned = nt.normalize(text)
    
    tokens = underthesea.word_tokenize(text_cleaned)
    
    tokens_without_stopwords = remove_stopwords(tokens)
    
    text_processed = ' '.join(tokens_without_stopwords)
    
    return text_processed

df['clean_content'] = df['content'].apply(preprocess_text_vietnamese_with_stopwords)

print(df[['content', 'clean_content']])

                                                 content  \
0      Bước vào trận đấu, Barcelona nhanh chóng tràn ...   
1      Willian đi vào lịch sử bóng đá xứ samba. Với 1...   
2      Giải vô địch ná cao su thế giới năm 2024 đã di...   
3      Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...   
4      Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...   
...                                                  ...   
98449  Tạo động lực phát triển nhanh và bền vững của ...   
98450  Theo số liệu của Tổng cục Hải quan, trong 9 th...   
98451  Mong có chiến lược ứng phó hiệu quả với thiên ...   
98452  Nhà chức trách xác định, Hoàng Văn Thảo có hàn...   
98453  Trong 6 tháng đầu năm 2024, Sở Y tế thành phố ...   

                                           clean_content  
0      trận đấu , barcelona nhanh chóng tràn tấn công...  
1      willian đi lịch sử bóng đá xứ samba . bàn thắn...  
2      giải vô địch ná cao su thế giới diễn hai , / t...  
3      mục tiêu phó thủ tướng chính phủ lê 

In [22]:
df.to_csv('clean_content_1.csv', index=False)

### Sử dụng tiếng việt từ bộ unicode: https://vietunicode.sourceforge.net/charset/vietalphabet.html

In [23]:
def clean_special_characters(text):        
    text = re.sub(r'[^a-zA-ZaAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'http[s]?://\S+', '', text)
    
    return text

In [24]:
df['clean_content'] = df['clean_content'].apply(clean_special_characters)

In [46]:
df['clean_content'].iloc[3]

'mục tiêu phó thủ tướng chính phủ lê thành long ký quyết định qđttg phê duyệt chiến lược phát triển thể dục thể thao việt nam tầm chiến lược mục tiêu xây dựng thể dục thể thao phát triển bền vững chuyên nghiệp người dân tiếp cận thụ hưởng dịch vụ thể dục thể thao nâng sức khỏe thể lực chất lượng cuộc sống nâng thành tích thể thao việt nam tiệm cận tiến ngang tầm thể thao phát triển châu á mở rộng thị trường thể thao huy động hiệu quả nguồn lực phát triển sự nghiệp thể dục thể thao chiến lược mục tiêu thể thao thành tích thường xuyên duy trì top kỳ sea games top kỳ asiad top kỳ olympic bóng đá nam top châu á giành quyền tham dự world cup bóng đá nữ top châu á giành quyền tham dự kỳ world cup mạng lưới cơ sở thể thao quốc gia hiện đại đăng cai asiad tỉnh tp trực thuộc t công trình thể thao cơ bản tiêu chuẩn tổ chức thi đấu quốc tế hành chính huyện công trình thể thao cơ bản đáp ứng tiêu chí tiêu chuẩn quy định lộ trình rõ ràng lịch sử việt nam hai đăng cai tổ chức đại hội thể thao đông n

In [26]:
df['tags'].iloc[3]   

'đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục,Quyết định số 1189/QĐ-TTg,phấn đấu,OCA,Đại hội Thể thao Đông Nam Á,chiến lược phát triển,Asian Indoor Games 2009,Incheon 2014,Hội đồng Olympic châu Á,ASIAD 18,Cung thể thao dưới nước,thể thao,Đại hội thể thao trong nhà châu Á,tầm nhìn,SEA Games,Đại hội Thể thao Đông Nam Á 2003,Lê Thành Long'

In [28]:
df.to_csv('clean_content_2.csv', index=False)  

# Xây dựng Đặc trưng
## Biểu diễn văn bản thành véc-tơ
* Sử dụng các kỹ thuật như TF-IDF
* Sử dụng các mô hình nhúng từ (Words Embeddings)

In [8]:
corpus = df['clean_content']
corpus

0        trận đấu barcelona nhanh chóng tràn tấn công v...
1        willian đi lịch sử bóng đá xứ samba bàn thắng ...
2        giải vô địch ná cao su thế giới diễn hai thượn...
3        mục tiêu phó thủ tướng chính phủ lê thành long...
4        giải bơi lặn vô địch quốc gia kết thúc đà nẵng...
                               ...                        
98449    động lực phát triển bền vững kinh tế nhiệm vụ ...
98450    số liệu tổng cục hải quan đầu xuất khẩu cao su...
98451    mong chiến lược ứng phó hiệu quả thiên tai kỳ ...
98452    nhà chức trách xác định hoàng văn thảo hành vi...
98453    đầu sở y tế thành phố đà nẵng triển khai gói t...
Name: clean_content, Length: 98454, dtype: category
Categories (98238, object): ['a h sinh trú thôn tê pen xã văn lem huyện đăk..., 'a hiêng sn trú thôn tê pan xã văn lem huyện đ..., 'a i sgtt sạch da đầu dầu gội tóc yếu hư hỏng ..., 'a khoản điều luật đất đai hợp đồng chuyển như..., ..., 'ứng viên vô địch đá muộn tâm đối đầu hà nội f..., 'ứng viên 

In [9]:
tfidf = TfidfVectorizer(max_features=3000)

In [10]:
tfidf

TfidfVectorizer(max_features=3000)

In [11]:
# Build Features
Xfeatures = tfidf.fit_transform(corpus).toarray()

In [12]:
Xfeatures

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.02377517],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0460459 ]])

In [ ]:
# for tag in list_5000_tags:
#     tag = str(tag)
#     df[tag] = df['tags'].apply(lambda x: 1 if tag in x else 0)

# df

In [14]:
df.dtypes

content          category
tags             category
clean_content    category
dtype: object

In [ ]:
# columns_to_select = df.columns[3:3003]  # Lấy các cột từ chỉ mục 2 trở đi
# y = df[columns_to_select]

In [13]:
y

,năm,Israel,Liên bang Nga,TP.HCM,bão,điểm chuẩn,xét tuyển,học sinh,giá vàng,Ukraine,...,hình thức,Vietnam,thành viên,Đông Nam Bộ,chuyện tình,Daniil Medvedev,tiền điện,vượt khó,chia cắt,nhà trọ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98449,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98450,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# y.to_csv('multilabel.csv', index=False)  

In [14]:
print(Xfeatures.shape)
print(y.shape)

(98454, 3000)
(98454, 3000)


In [15]:
# Split Data 
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.2,random_state=42)

In [16]:
print(df['clean_content'].shape)
print(X_train.shape)

(98454,)
(78763, 3000)


In [17]:
### Problem Transform
import skmultilearn
dir(skmultilearn)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'adapt',
 'base',
 'problem_transform',
 'utils']

# Binary Relevance classficiation

In [18]:
# Convert Our Multi-Label Prob to Multi-Class
# binary classficiation
binary_rel_clf = BinaryRelevance(MultinomialNB())

In [19]:
binary_rel_clf.fit(X_train,y_train)

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [ ]:
# # Sử dụng cross-validation với 5 folds và đánh giá bằng F1-micro
# scores = cross_val_score(binary_rel_clf, X_train, y_train, cv=5, scoring='f1_micro')

# # In kết quả
# print("F1-micro scores for each fold: ", scores)
# print("Mean F1-micro: ", np.mean(scores))                                                                  

In [20]:
# Predictions
br_prediction = binary_rel_clf.predict(X_test)

In [21]:
br_prediction 

<Compressed Sparse Column sparse matrix of dtype 'int8'
	with 166631 stored elements and shape (19691, 3000)>

In [22]:
# Convert to Array  To See Result
br_prediction.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [27]:
# Accuracy
accuracy = accuracy_score(y_test,br_prediction)

# Tính F1-micro
f1_micro = f1_score(y_test, br_prediction, average='micro')

print("Accuracy: ", accuracy)
print("F1 Micro: ", f1_micro)

Accuracy:  0.00010156924483266467
F1 Micro:  0.32367003298951474


In [28]:
# Hamming Loss :Incorrect Predictions
# The Lower the result the better
hamming_loss(y_test,br_prediction)

0.005098183603338242

In [ ]:
binary_rel_clf = BinaryRelevance(MultinomialNB())

# Cross-validation với 5 folds và đánh giá bằng F1-micro

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(binary_rel_clf, X_train, y_train, cv=kf, scoring='f1_micro')
print(f'F1-micro scores from cross-validation: {scores}')
print(f'Mean F1-micro score: {scores.mean()}')

# Huấn luyện lại mô hình trên toàn bộ dữ liệu huấn luyện
binary_rel_clf.fit(X_train, y_train)

# Dự đoán trên tập test
y_pred = binary_rel_clf.predict(X_test)

# Tính F1-micro trên tập test
f1_test = f1_score(y_test, y_pred, average='micro')
print(f'F1-micro score on the test set: {f1_test}')

# Classifier Chains

In [ ]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    # Predict
    clf_predictions = clf.predict(xtest)
    # Check For Accuracy
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

In [ ]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)
clf_chain_model


# Label Powerset


In [ ]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)

In [ ]:
clf_labelP_model

In [ ]:
ex1 = df['title'].iloc[0]

In [ ]:
# Vectorized 
vec_example = tfidf.transform([ex1])

In [ ]:
# Make our prediction
binary_rel_clf.predict(vec_example).toarray()

# Lưu và chạy lại

In [29]:
import joblib

In [30]:
# joblib.dump(binary_rel_clf, 'binary_rel_clf_model_1.pkl')


# Save Model
binary_rel_clf_file = open("binary_rel_clf_model_file_1.pkl","wb")
joblib.dump(binary_rel_clf,binary_rel_clf_file)
binary_rel_clf_file.close()
print("Mô hình đã được lưu!")

Mô hình đã được lưu!


In [31]:
# Save Vectorizer
tfidf_vectorizer_file = open("tfidf_vectorizer_SO_tags_file.pkl","wb")
joblib.dump(tfidf,tfidf_vectorizer_file)
tfidf_vectorizer_file.close()

In [ ]:
# Tải lại mô hình đã lưu
binary_rel_clf_loaded = joblib.load('binary_rel_clf_model_1.pkl')

# Tiếp tục huấn luyện mô hình với dữ liệu mới (nếu có)
binary_rel_clf_loaded.fit(X_train, y_train)